In [59]:
import pandas as pd
import csv
import random

In [60]:
def make_quote(l, idx):
    return '"' + l[idx] + '"'

def make_write_str(l, join_str=',\n', double_new_line=True):
    ret = '\n(' + join_str.join(l) + ')'
    if double_new_line:
        return ret + '\n'
    return ret

def make_real_list(l):
    l = l[1:-1] # removes the '[' and ']'
    return l.split(', ')

In [61]:
random.seed(3500)
with open('food_network_recipes_1.csv', 'r') as read_file:
    reader = csv.reader(read_file)
    with open('recipe_inserts.sql', 'a+') as write_file:
        write_file.write('INSERT INTO Recipe (recipe_id, title, minutes, servings, instructions, user_id)')
        header = next(reader)
        for line in reader:
            line[1] = make_quote(line, 1)
            line[5] = make_quote(line, 5)
            line.pop(2)
            line.append(str(random.randint(0,4)))
            write_str = make_write_str(line)
            write_file.write(write_str)
            
        write_file.write(';')
            

In [62]:
with open('food_network_ingredients_1.csv', 'r') as read_file2, \
     open('ingredient_inserts.sql', 'a+') as ing_file, \
     open('ri_inserts.sql', 'a+') as ri_file:
    reader = csv.reader(read_file2)
    ing_file.write('INSERT INTO Ingredient (ingredient_name)')
    ri_file.write('INSERT INTO Recipe_ingredient (ingredient_id, recipe_id, amount, unit, optional)')
    next(reader)
    
    ing_id_counter = 0
    ing_dict = {}
    
    for line in reader:
        recipe_id = line[0]
        ing_list = make_real_list(line[1])
        amount_list = make_real_list(line[2])
        unit_list = make_real_list(line[3])
        option_list = make_real_list(line[4])
        
        for i in range(len(ing_list)):
            if ing_list[i] not in ing_dict:
                ing_file.write('\n(' + ing_list[i] + ')')
                ing_dict[ing_list[i]] = ing_id_counter
                ing_id = str(ing_id_counter)
                ing_id_counter += 1
            else:
                ing_id = str(ing_dict[ing_list[i]])
            write_str = make_write_str([ing_id, recipe_id, amount_list[i], unit_list[i], option_list[i]], join_str=', ', double_new_line=False)
            ri_file.write(write_str)
            
    ri_file.write(';')
    ing_file.write(';')
        
    
    